# Анализ данных на Python

*Алла Тамбовцева*

## Практикум по иерархическому кластерному анализу (количественные данные)

В этом практикуме мы будем работать с данными по районам Балтимора из файла `Baltimore_data.csv`. В этом файле собраны показатели, которые можно считать характеристиками благополучия района, в том числе с точки зрения покупки в этом районе жилья:

* `CSA2010`: название укрупненного района, по которому ведётся сбор статистических данных (*Community Statistical Area*), в соответствии с делением в 2010 году;
* `trees17`: процент деревьев в районе по состоянию на 2017 год (подробнее [здесь](https://data.baltimorecity.gov/maps/e8b7beca0fd649b1a77c58fafc4658a9/about));
* `racdiv21`: индекс расового/этнического разнообразия за 2021 год, более высокие значения – более высокое разнообразие (подробнее [здесь](https://data.baltimorecity.gov/maps/d588f7de06cf4815951e105bb8a390b1/about));
* `viol21`: число тяжких преступлений на 1000 жителей в 2021 году, включает число убийств, изнасилований, нападения, грабежи (подробнее [здесь](https://data.baltimorecity.gov/maps/ab03385abf3b4f50aec0b090caa8877a/about));
* `salepr19`: медианная цена продажи жилья в районе за 2019 год.

Глобальные задачи:

* поделить районы на группы с помощью иерархического кластерного анализа;
* посмотреть, связано ли деление на группы с географическим расположением районов.

### Часть 1: загрузка данных и разведывательный анализ

Импортируем необходимые библиотеки:

In [ ]:
import warnings

import pandas as pd
import numpy as np

# выключаем предупреждения
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

Загрузим данные из CSV-файла: 

In [ ]:
df = pd.read_csv("Baltimore_data.csv")
df.head()

Уберём лишний столбец *Unnamed: 0*:

In [ ]:
df = df.drop(columns = ["Unnamed: 0"])
df.head()

Выведем описательные статистики для всех столбцов датафрейма:

In [ ]:
df.describe()

Построим гистограммы для всех числовых показателей, чтобы понять, можно ли уже на этом этапе, рассматривая переменные по отдельности, выделить какие-то группы и сделать предположения о том, сколько кластеров будет логично ожидать. Начнем с процента деревьев в районе:

In [ ]:
df["trees17"].hist(color = "forestgreen", edgecolor = "white");

Посмотрим на расовое/этническое разнообразие:

In [ ]:
df["racdiv21"].hist(color = "grey", edgecolor = "white");

Изучим преступность:

In [ ]:
df["viol21"].hist(color = "black", edgecolor = "white");

Тут интересная история. С одной стороны, есть районы с низкой преступностью (10-15 преступлений на 1000 жителей) и более высокой преступностью (30-35 преступлений на 1000 жителей). С другой стороны, есть какой-то район с запредельным, по сравнению с остальными, уровнем преступности. Кто бы это мог быть?

In [ ]:
### выясним

Осталось изучить цены на жильё:

In [ ]:
df["salepr19"].hist(color = "gold", edgecolor = "white");

Теперь попробуем рассмотреть совместное распределение. Построим диаграмму рассеивания между преступностью и медианной стоимостью жилья:

In [ ]:
df.plot.scatter(x = "viol21", y = "salepr19");

Теперь давайте построим диаграмму рассеивания с дополнительными переменными – третьим измерением в виде цвета:

In [ ]:
df.plot.scatter(x = "viol21", y = "salepr19", c = "trees17");

С таким же успехом в качестве третьего измерения можно добавить и индекс расового/этнического разнообразия:

In [ ]:
df.plot.scatter(x = "viol21", y = "salepr19", c = "racdiv21");

### Часть 2: иерархический кластерный анализ

Итак, мы выяснили, что на этапе разведывательного анализа данных выделить какие-то явные группы районов нам не удалось. Однако мы точно видели, что основания для деления есть, как минимум, понятно, что:

* есть районы с высокой преступностью и одновременно более низкой стоимостью жилья, 
* есть достаточно благополучные дорогие районы, при этом они могут быть более зелёными (возможно, окраины города с парками) и менее зелёными (возможно, центральные районы с плотной застройкой, но не такие неоднозначные как Даунтаун).

Перейдём к кластерному анализу. Для начала выберем только числовые столбцы – текстовые будут нам мешать, поскольку данные нужно будет стандартизировать, а затем считать по ним расстояния. Если бы они были нужны, мы бы их перекодировали, но это отдельная история, мы пока работаем только с количественными данными. 

In [ ]:
to_clust = df.select_dtypes(include = float)
to_clust.head()

Текстовый столбец с названием районов мы убрали, но саму информацию о том, где какой район, хотелось бы сохранить, иначе не совсем понятно, как в дальнейшем интерпретировать результаты кластеризации. 

Добавим текстовые метки в качестве названий строк:

In [ ]:
to_clust.index = df["CSA2010"]
to_clust.head()

Выполним шкалирование (центрирование и нормирование) данных. Посчитаем по каждому столбцу среднее и стандартное отклонение:

In [ ]:
to_clust.mean()

In [ ]:
to_clust.std()

Вычтем из каждого значения столбца соответствующее среднее и поделим на стандартное отклонение (что удобно, `pandas` позволяет проделать это сразу для всех столбцов датафрейма):

In [ ]:
scaled_data = (to_clust - to_clust.mean()) / to_clust.std()
scaled_data.head()

Наконец переходим к иерархическому кластерному анализу. Импортируем из библиотеки `scipy` (*Scientific Python*) модуль `cluster`, из него набор функций `hierarchy`, а из него уже выбираем функции `linkage`, `dendrogram` и `cut_tree`:

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, cut_tree

Одним из самых надёжных методов агрегирования – метод Варда (Уорда), он требует использования квадрата евклидова расстояния (на самом деле, если аргумент `metric` не указывать, всё равно результат будет корректным – Python и так знает, что использовать, если в `method` указано `ward`):

In [ ]:
ward_clustering = linkage(scaled_data, method = "ward", metric = "euclidean")

Построим дендрограмму:

In [ ]:
dendrogram(ward_clustering);

Согласимся с Python и выделим три кластера – три «ветки» наблюдений. Сохраним соответствующие метки кластеров в датафрейм:

In [ ]:
cluster_labels = cut_tree(ward_clustering, n_clusters = 3).reshape(-1, )
df["cluster"] = cluster_labels
df.head()

**Дополнительно – для понимания кода выше.** Функция `cut_tree()` «разрезает» дендрограмму на желаемое число кластеров `n_clusters` (можно «разрезать» и по расстоянию `height`) и возвращает массив с метками кластеров такого вида:

In [ ]:
# кластеры 0, 1, 2
cut_tree(ward_clustering, n_clusters = 3)

Вокруг каждой числовой метки есть лишние квадратные скобки. Чтобы от них избавиться, вызываем метод `.reshape()`, который позволяет перегруппировать массивы. В общем случае внутри `.reshape()` можно указывать два элемента размерности массива – число строк и столбцов:

In [ ]:
# например, массив из 55 элементов сделали массивом размерности 5 на 11
# 5 строк (5 списков) и 11 столбцов (11 элементов в каждом списке)

cut_tree(ward_clustering, n_clusters = 3).reshape(5, 11)

В изначальном массиве были лишние скобки вокруг «строк» – посмотрим на размерность:

In [ ]:
cut_tree(ward_clustering, n_clusters = 3).shape

Число 1 здесь можно интерпретировать как число списков в каждой строке. Мы хотим от этих списков избавиться, чтобы получить обычный перечень значений, поэтому в `.reshape()` указываем значение `-1`. В итоге получается массив размерности 55 на 0, что в рамках NumPy соответствует одномерным массивам, похожим на обычные списки в Python:

In [ ]:
cut_tree(ward_clustering, n_clusters = 3).reshape(-1, ).shape

In [ ]:
# внутри array() – просто список без лишних скобок вокруг элементов

cut_tree(ward_clustering, n_clusters = 3).reshape(-1, )

Вернёмся к полученным кластерам, чьи названия мы сохранили в столбец `cluster`. Посмотрим, сколько наблюдений в каждом кластере:

In [ ]:
df["cluster"].value_counts()

Кластер с индексом 2 – самый большой, с индексом 0 – самый маленький. Но при этом число наблюдений в каждом кластере не супер-маленькое, явных оснований для укрупнений (или для отдельного изучения этих «микрокластеров» нет). 

Посмотрим на средние по группам:

In [ ]:
# в .groupby() указываем основание для группировки
# в .agg() указываем функцию для нужной статистики
# numeric_only – считаем средние только для числовых столбцов

df.groupby("cluster").agg("mean", numeric_only = True)

Итак, средние значения показателей в трёх группах, чисто визуально, различаются, причём в большинстве случаев довольно существенно. Для наглядности давайте перейдём к визуализации распределений по группам и построим ящики с усами.

In [ ]:
df.boxplot(column = "trees17", by = "cluster");

In [ ]:
df.boxplot(column = "racdiv21", by = "cluster");

In [ ]:
df.boxplot(column = "viol21", by = "cluster");

In [ ]:
df.boxplot(column = "salepr19", by = "cluster");

### Часть 3: кластеры и география

Установим и импортируем библиотеку `geopandas`, это надстройка над `pandas`, которая позволяет загружать файлы с географической информацией (в частности, файлы `.geojson`) и отрисовывать карты. 

In [ ]:
# раскомментируйте строку ниже для установки
#!pip install geopandas

In [ ]:
import geopandas

В файле `Percent_of_Area_Covered_by_Trees.geojson` хранится информация о проценте деревьев в районах и вспомогательная географическая информация о каждом районе – набор точек для определения и отрисовки границ района. 

При загрузке geojson-файла через функцию `read_file()` данные внешне ничем не отличаются от обычного датафрейма `pandas`:

In [ ]:
gdf = geopandas.read_file("Percent_of_Area_Covered_by_Trees.geojson")
gdf.head()

Каждый район – это отдельный объект на карте. Чисто геометрически, этот объект – многоугольник, то есть какая-то область, ограниченная замкнутой ломаной линией, *polygon* на английском языке. Поэтому здесь в таблице в столбце `geometry` хранятся объекты специального типа *POLYGON*, которые внутри похожи на кортежи с парными координатами точек (широта и долгота) в выбранной географической проекции. По этим точкам район отрисовывается на карте. В столбце `geometry` также есть объекты типа *MULTIPOLYGON* для больших районов или районов со сложными границами, которые удобнее собрать из нескольких многоугольников. Столбец `Shape__Area` – это площадь района на карте, площадь многоугольника, а столбец `Shape__Length` – его периметр, длина ограничивающей замкнутой линии. 

К такому более продвинутому датафрейму типа *GeoDataFrame* можно применить метод `.plot()` и построить карту!

In [ ]:
# выставляем размер 16 на 9 дюймов – побольше

gdf.plot(figsize = (16, 9));

Если на вход методу `.plot()` мы ещё подадим столбец с какими-то текстовыми или числовыми метками, произойдёт автоматическая раскраска карты:

In [ ]:
gdf.plot("CSA2010", figsize = (16, 9));

Вообще каждому уникальному значению `CSA2010`, то есть каждому району, должен соответствовать уникальный цвет, но поскольку таких больших готовых палитр на 55 цветов не существует, с какого-то момента Python начинает использовать те же цвета. 

Добавим в `gdf` столбец с метками наших кластеров:

In [ ]:
gdf["cluster"] = df["cluster"]

In [ ]:
gdf.plot("cluster", figsize = (16, 9));

Выглядит получше, но не совсем с ходу понятно, где какой кластер. Давайте создадим копию столбца с метками кластеров, назовём его `color`, а затем в виде словаря опишем пары соответствий – какие значения на какой цвет поменять:

In [ ]:
# создаем копию столбца
gdf["color"] = gdf["cluster"]

# словарь для перевода метки в цвета
convert = {0 : "darkred", 1: "forestgreen", 2 : "darkblue"}

# замена по правилу, описанном в словаре 
gdf = gdf.replace({"color" : convert})

# раз свои цвета – не просто color, а color = gdf["color"]
gdf.plot(color = gdf["color"], figsize = (16, 9));

Итого: 

* кластер 1 (темно-зелёный цвет) с самыми зелёными районами – северная часть города;
* кластер 0 (тёмно-красный цвет) с не самыми благополучными районами – «непарадная» часть города, не центр и не районы вокруг Внутренней Гавани;
* кластер 2 (тёмно-синий цвет) с благополучными незелёными районами – преимущественно «парадная» часть города, исторический и деловой центр, районы с развитой инфраструктурой и проч.

P.S. Глядя на карту, возникает вопрос: а что это за таинственный белый квадрат в середине карты? Обычно такое возникает либо в случае, когда данных нет (даже географическая информация по району отсутствует), либо когда на данном месте находится что-то незаселённое (водоём, например). Можете провести самостоятельное расследование ([онлайн-карта](https://data.baltimorecity.gov/maps/e8b7beca0fd649b1a77c58fafc4658a9/about) по тому же исходнику вам в помощь)!